In [2]:
import tensorflow as tf
import keras
import numpy as np
import torch.nn.functional as F
from keras.datasets import mnist
import torch
(train_X, train_y), (test_X, test_y) = keras.datasets.cifar10.load_data()
Xtrain = torch.tensor(train_X).float().permute((0,3,1,2))
Xtest = torch.tensor(test_X[:5000]).float().permute((0,3,1,2))
Xval = torch.tensor(test_X[5000:]).float().permute((0,3,1,2))
train_y = train_y.reshape(train_y.shape[0])
test_y = test_y.reshape(test_y.shape[0])
Ytrain = torch.tensor(train_y, dtype= torch.long)
Ytest = torch.tensor(test_y[:5000], dtype= torch.long)
Yval = torch.tensor(test_y[5000:], dtype= torch.long)

170498071/170498071 [==============================] - 3s 0us/step


In [3]:
mean_image = torch.mean(Xtrain, axis = 0)
Xtrain -= mean_image
Xtest -= mean_image
Xval -= mean_image

In [28]:
def affine_forward(x, w, b):
  x_reshaped = x.reshape(x.shape[0], -1)
  out = x_reshaped @ w + b
  return out

def relu_forward(x):
  zero = torch.tensor(0).float()
  return torch.maximum(x, zero)

def conv_forward(x, w, b, conv_param):
  N, C, H, W = x.shape
  A, C, f, f = w.shape
  stride = conv_param['stride']
  pad = conv_param['pad']
  n_H = 1 + (H + 2 * pad - f) // stride
  n_W = 1 + (W + 2 * pad - f) // stride
  out_unf = (F.unfold(x, (f,f), padding=pad, stride=stride).transpose(1,2) @ w.view(A, -1).t()).transpose(1,2)
  out = out_unf.view(N, A, n_H, n_W) + b
  return out

def pool_forward(x, hparam):
  m, C, H, W = x.shape
  f = hparam['f']
  stride = hparam['stride']
  input_windows = x.unfold(2, f, stride).unfold(3,f, stride)
  input_windows = input_windows.contiguous().view(*input_windows.size()[:-2], -1)
  max_val, max_idx = input_windows.max(4)
  return max_val


In [32]:
act = 32
f = 7
stride = 1
W1 = (torch.randn((act,3,f,f)) * 0.01).requires_grad_()
b1 = torch.zeros((1,act,1,1),requires_grad = True)
W2 = (torch.randn((8192, 100)) * 0.01).requires_grad_()
b2 = torch.zeros((100),requires_grad = True)
W3 = (torch.randn((100, 10)) * 0.01).requires_grad_()
b3 = torch.zeros((10),requires_grad = True)
parameters = [W1,b1,W2,b2, W3, b3]

conv_param = {'stride': stride, 'pad': (f - stride)//2}

hparam = {'stride': 2, 'f': 2}


v = dict()
n = len(parameters)
for i in range(n):
  v[f'v{i}'] = v.get(f'v{i}', 0)

In [51]:
mu = 0.9
for i in range(2000):
  ix = torch.randint(0, Xtrain.shape[0], (150,))
  C1 = conv_forward(Xtrain[ix], W1, b1, conv_param)
  P1 = pool_forward(C1, hparam)
  A1 = relu_forward(P1)
  Z1 = affine_forward(A1, W2, b2)
  A2 = relu_forward(Z1)
  Z2 = affine_forward(A2, W3, b3)
  loss = F.cross_entropy(Z2, Ytrain[ix])
  print(loss)
  for p in parameters:
    p.grad = None
  loss.backward()
  lr = 0.001
  for n, p in enumerate(parameters):
    v[f'v{n}'] = mu * v[f'v{n}'] - lr * p.grad
    p.data += v[f'v{n}']

tensor(0.5810, grad_fn=<NllLossBackward0>)
tensor(0.6243, grad_fn=<NllLossBackward0>)
tensor(0.7518, grad_fn=<NllLossBackward0>)
tensor(0.6786, grad_fn=<NllLossBackward0>)
tensor(0.5511, grad_fn=<NllLossBackward0>)
tensor(0.7579, grad_fn=<NllLossBackward0>)
tensor(0.6753, grad_fn=<NllLossBackward0>)
tensor(0.6081, grad_fn=<NllLossBackward0>)
tensor(0.6028, grad_fn=<NllLossBackward0>)
tensor(0.6674, grad_fn=<NllLossBackward0>)
tensor(0.5543, grad_fn=<NllLossBackward0>)
tensor(0.5864, grad_fn=<NllLossBackward0>)
tensor(0.6442, grad_fn=<NllLossBackward0>)
tensor(0.6167, grad_fn=<NllLossBackward0>)
tensor(0.7663, grad_fn=<NllLossBackward0>)
tensor(0.5123, grad_fn=<NllLossBackward0>)
tensor(0.5291, grad_fn=<NllLossBackward0>)
tensor(0.5848, grad_fn=<NllLossBackward0>)
tensor(0.6579, grad_fn=<NllLossBackward0>)
tensor(0.6168, grad_fn=<NllLossBackward0>)
tensor(0.4955, grad_fn=<NllLossBackward0>)
tensor(0.6359, grad_fn=<NllLossBackward0>)
tensor(0.5991, grad_fn=<NllLossBackward0>)
tensor(0.63

KeyboardInterrupt: 

In [52]:
from sklearn.metrics import accuracy_score
sample_size = 32
ix = torch.randint(0, Xtest.shape[0], (32,))
C1 = conv_forward(Xtest[ix], W1, b1, conv_param)
P1 = pool_forward(C1, hparam)
A1 = relu_forward(P1)
Z1 = affine_forward(A1, W2, b2)
A2 = relu_forward(Z1)
scores = affine_forward(A2, W3, b3)

predictions = []
for i in range(32):
  predictions.append(torch.argmax(scores[i]))
accuracy = accuracy_score(predictions,Ytest[ix])
print(accuracy)
print(predictions)

0.5
[tensor(8), tensor(8), tensor(7), tensor(7), tensor(8), tensor(3), tensor(1), tensor(2), tensor(9), tensor(4), tensor(7), tensor(7), tensor(5), tensor(0), tensor(3), tensor(5), tensor(9), tensor(5), tensor(3), tensor(1), tensor(5), tensor(4), tensor(2), tensor(1), tensor(8), tensor(2), tensor(2), tensor(5), tensor(5), tensor(3), tensor(3), tensor(3)]
